In [6]:
!pip install -q uagents google-genai

In [ ]:
import os, time, multiprocessing, asyncio
import nest_asyncio
from google import genai
from pydantic import BaseModel, Field
from uagents import Agent, Context

nest_asyncio.apply()

In [ ]:
os.environ["GOOGLE_API_KEY"] = "Use Your Own API Key Here"

client = genai.Client()

In [ ]:
class Question(BaseModel):
    question: str = Field(...)

class Answer(BaseModel):
    answer: str = Field(...)

In [ ]:
ai_agent = Agent(
    name="gemini_agent",
    seed="agent_seed_phrase",
    port=8000,
    endpoint=["http://127.0.0.1:8000/submit"]
)

@ai_agent.on_event("startup")
async def ai_startup(ctx: Context):
    ctx.logger.info(f"{ai_agent.name} listening on {ai_agent.address}")

def ask_gemini(q: str) -> str:
    resp = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=f"Answer the question: {q}"
    )
    return resp.text

@ai_agent.on_message(model=Question, replies=Answer)
async def handle_question(ctx: Context, sender: str, msg: Question):
    ans = ask_gemini(msg.question)
    await ctx.send(sender, Answer(answer=ans))

In [ ]:
client_agent = Agent(
    name="client_agent",
    seed="client_seed_phrase",
    port=8001,
    endpoint=["http://127.0.0.1:8001/submit"]
)

@client_agent.on_event("startup")
async def ask_on_start(ctx: Context):
    await ctx.send(ai_agent.address, Question(question="What is the capital of France?"))

@client_agent.on_message(model=Answer)
async def handle_answer(ctx: Context, sender: str, msg: Answer):
    print("📨 Answer from Gemini:", msg.answer)
    # Use a more graceful shutdown
    asyncio.create_task(shutdown_loop())

async def shutdown_loop():
    await asyncio.sleep(1)  # Give time for cleanup
    loop = asyncio.get_event_loop()
    loop.stop()

In [ ]:
def run_agent(agent):
    agent.run()

if __name__ == "__main__":
    p = multiprocessing.Process(target=run_agent, args=(ai_agent,))
    p.start()
    time.sleep(2)

    client_agent.run()

    p.join()

INFO:     [gemini_agent]: Starting agent with address: agent1qvsw5cmxhvm5de9a45ens4572srfjvtdsp67ewnjlrw2kre9wj0suq5k6ve
INFO:     [gemini_agent]: Starting agent with address: agent1qvsw5cmxhvm5de9a45ens4572srfjvtdsp67ewnjlrw2kre9wj0suq5k6ve
INFO:     [gemini_agent]: gemini_agent listening on agent1qvsw5cmxhvm5de9a45ens4572srfjvtdsp67ewnjlrw2kre9wj0suq5k6ve
INFO:     [gemini_agent]: gemini_agent listening on agent1qvsw5cmxhvm5de9a45ens4572srfjvtdsp67ewnjlrw2kre9wj0suq5k6ve
INFO:     [gemini_agent]: Agent inspector available at https://agentverse.ai/inspect/?uri=http%3A//127.0.0.1%3A8000&address=agent1qvsw5cmxhvm5de9a45ens4572srfjvtdsp67ewnjlrw2kre9wj0suq5k6ve
INFO:     [gemini_agent]: Agent inspector available at https://agentverse.ai/inspect/?uri=http%3A//127.0.0.1%3A8000&address=agent1qvsw5cmxhvm5de9a45ens4572srfjvtdsp67ewnjlrw2kre9wj0suq5k6ve
INFO:     [gemini_agent]: Starting server on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     [gemini_agent]: Starting server on http://0.